<a href="https://colab.research.google.com/github/KChamroeun/Application/blob/main/CalendarRender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

heavenly_stem_characters = {
    "Jia": "甲 ឈើ+",
    "Yi": "乙 ឈើ-",
    "Bing": "丙 ភ្លើង+",
    "Ding": "丁 ភ្លើង-",
    "Wu": "戊 ដី+",
    "Ji": "己 ដី-",
    "Geng": "庚 មាស+",
    "Xin": "辛 មាស-",
    "Ren": "壬 ទឹក+",
    "Gui": "癸 ទឹក-"
      }
heavenly_stem_characters_for_year = {
    0: "庚 Geng មាស+",
    1: "辛 Xin 庚 មាស-",
    2: "壬 Ren ទឹក+",
    3: "癸 Gui ទឹក-",
    4: "甲 Jia ឈើ+",
    5: "乙 Yi ឈើ-",
    6: "丙 Bing ភ្លើង+",
    7: "丁 Ding ភ្លើង-",
    8: "戊 Wu ដី+",
    9: "己 Ji ដី-"
}

print(heavenly_stem_characters_for_year)

print(heavenly_stem_characters)





{'Jia': '甲 ឈើ+', 'Yi': '乙 ឈើ-', 'Bing': '丙 ភ្លើង+', 'Ding': '丁 ភ្លើង-', 'Wu': '戊 ដី+', 'Ji': '己 ដី-', 'Geng': '庚 មាស+', 'Xin': '辛 មាស-', 'Ren': '壬 ទឹក+', 'Gui': '癸 ទឹក-'}


Render Calendar based on Lichun and Winter solstice

In [ ]:
!pip install astronomy
!pip install Astronomy

Winter Solstic on 21 or 22nd using skyfield library

| Library    | Accuracy   | Size / Setup   | Best for…                  |
| ---------- | ---------- | -------------- | -------------------------- |
| `astral`   | ±1 min     | tiny, no files | Quick calendar apps        |
| `skyfield` | ±ms        | 6 MB kernel    | Precision, long time spans |
| `pyephem`  | deprecated | —              | Use Skyfield instead       |


“Skyfield” is Python‑only — but you still have good options in Flutter / Dart


| Approach                                                     | How it works                                                                                                                                           | Pros                                                                                    | Cons                                                                                          |
| ------------------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------- |
| **1 · Call a small Python micro‑service that runs Skyfield** | Your Flutter app makes HTTPS requests to a FastAPI / Flask / Cloud Function that returns the solstice day (21 or 22 Dec) for a given year.             | • Zero astronomical math to port<br>• Always up‑to‑date if you keep the service current | • Requires hosting a serverless function or VM<br>• Adds network latency / offline dependency |
| **2 · Port the minimal math to Dart**                        | Skyfield ultimately solves *solar ecliptic longitude = 270°.* You can implement the same Meeus algorithms (or use an existing Dart astronomy library). | • Pure Dart (no backend)<br>• Works offline, 100 % client‑side                          | • You must write / port \~150 lines of math or adopt a 3rd‑party lib                          |
| **3 · Use a ready‑made Dart astronomy package**              | `package:astro` and `package:astronomy` exist on pub.dev; both compute solar positions.                                                                | • One‑liner API similar to Skyfield                                                     | • Smaller community than Skyfield; verify accuracy                                            |


In [ ]:
!pip install skyfield --quiet
!pip install Astronomy --quiet

from skyfield.api import load
from skyfield import almanac
from datetime import timezone

ts = load.timescale()
eph = load('de440s.bsp')

def december_solstice(year):
    t0 = ts.utc(year, 12, 19)
    t1 = ts.utc(year, 12, 24)
    times, events = almanac.find_discrete(t0, t1, almanac.seasons(eph))
    for t, e in zip(times, events):
        if e == 3:  # 3 means December solstice
            return t.utc_datetime().replace(tzinfo=timezone.utc)

print(december_solstice(2015))


2015-12-22 04:47:56.661933+00:00


In [ ]:
from datetime import date

def get_solstice_cutoff(year: int) -> tuple:
    """
    Return the correct solstice cutoff for a given year.
    Defaults to Dec 21 unless listed as a Dec 22 solstice year.
    """
    solstice_22_years = {
        2023, 2027, 2031, 2034, 2038, 2042, 2046, 2049,
        2053, 2057, 2061, 2064, 2068, 2072, 2076
    }

    return (12, 22) if year in solstice_22_years else (12, 21)


| Term                | Solar Longitude | Typical Date | Skyfield Match |
| ------------------- | --------------- | ------------ | -------------- |
| **Lìchūn (立春)**     | 315°            | \~Feb 3–4    | `lon ≈ 315°`   |
| **Winter Solstice** | 270°            | \~Dec 21–22  | `lon ≈ 270°`   |


Lichun calendar

In [ ]:
!pip install skyfield --quit
from skyfield.api import load
from datetime import timedelta

# Load ephemeris data from JPL
eph = load('de421.bsp')
ts = load.timescale()
earth = eph['earth']
sun = eph['sun']

def get_solar_longitude(t):
    astrometric = earth.at(t).observe(sun).apparent()
    _, lon, _ = astrometric.ecliptic_latlon()
    return lon.degrees % 360

def find_lichun(year):
    t0 = ts.utc(year, 2, 2)
    t1 = ts.utc(year, 2, 5)

    t = t0
    step = timedelta(minutes=10)

    while t.utc_datetime() < t1.utc_datetime():
        lon = get_solar_longitude(t)
        if 314.9 <= lon <= 315.1:
            return t.utc_datetime()
        t = ts.utc(t.utc_datetime() + step)

    return None

# Example: Get Lichun for 2025
lichun_2025 = find_lichun(2025)
print("Lìchūn 2025:", lichun_2025)


Winter Solstice

In [ ]:
from skyfield.api import load
from datetime import timedelta

# Load ephemeris data
eph = load('de421.bsp')
ts = load.timescale()

earth = eph['earth']
sun = eph['sun']

# Get solar ecliptic longitude in degrees
def get_solar_longitude(t):
    astrometric = earth.at(t).observe(sun).apparent()
    _, lon, _ = astrometric.ecliptic_latlon()
    return lon.degrees % 360

# Find Winter Solstice (when longitude is 270°)
def find_winter_solstice(year):
    t0 = ts.utc(year, 12, 20)
    t1 = ts.utc(year, 12, 23)

    t = t0
    step = timedelta(minutes=10)

    while t.utc_datetime() < t1.utc_datetime():
        lon = get_solar_longitude(t)
        if 269.9 <= lon <= 270.1:
            return t.utc_datetime()
        t = ts.utc(t.utc_datetime() + step)

    return None

# Example usage
winter_solstice_2025 = find_winter_solstice(2025)
print("Winter Solstice 2025:", winter_solstice_2025)


“Skyfield” is Python‑only — but you still have good options in Flutter / Dart

| Approach                                                     | How it works                                                                                                                                           | Pros                                                                                    | Cons                                                                                          |
| ------------------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------- |
| **1 · Call a small Python micro‑service that runs Skyfield** | Your Flutter app makes HTTPS requests to a FastAPI / Flask / Cloud Function that returns the solstice day (21 or 22 Dec) for a given year.             | • Zero astronomical math to port<br>• Always up‑to‑date if you keep the service current | • Requires hosting a serverless function or VM<br>• Adds network latency / offline dependency |
| **2 · Port the minimal math to Dart**                        | Skyfield ultimately solves *solar ecliptic longitude = 270°.* You can implement the same Meeus algorithms (or use an existing Dart astronomy library). | • Pure Dart (no backend)<br>• Works offline, 100 % client‑side                          | • You must write / port \~150 lines of math or adopt a 3rd‑party lib                          |
| **3 · Use a ready‑made Dart astronomy package**              | `package:astro` and `package:astronomy` exist on pub.dev; both compute solar positions.                                                                | • One‑liner API similar to Skyfield                                                     | • Smaller community than Skyfield; verify accuracy                                            |
